In [ ]:
#r "nuget:System.Device.Gpio,1.3"
#r "nuget:IoT.Device.Bindings,1.3"
#r "nuget:UnitsNet,4.82.0"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package UnitsNet version 4.82.0

Installed package System.Device.Gpio version 1.3.0

Installed package IoT.Device.Bindings version 1.3.0

In [ ]:
#pragma warning disable CS1702
using Iot.Device.SenseHat;
using System;
using System.Drawing;
using (var ledMatrix = new SenseHatLedMatrixI2c())
{
        ledMatrix.Fill(Color.Black);
}

In [ ]:
async IAsyncEnumerable<double> ReadTemperatureAsync(int count = 10){   
    using (var sh = new SenseHat())
    {
        for (var i = 0; i < count; i++){
            yield return (sh.Temperature2.DegreesFahrenheit);
            await Task.Delay(60000);
        }
    };
}

In [ ]:
using System;
var temperatures = new List<(DateTime dt, double val)>();


In [ ]:
DateTime RoundUp(DateTime dt, TimeSpan d)
{
    return new DateTime((dt.Ticks + d.Ticks - 1) / d.Ticks * d.Ticks, dt.Kind);
}

In [ ]:
await foreach (var result in ReadTemperatureAsync(15))
{
    var t = RoundUp(DateTime.Now, TimeSpan.FromMinutes(1));
    Console.WriteLine(t + " : " + result);
    temperatures.Add((t, result));
}

2/23/2021 9:32:00 PM : 108.5
2/23/2021 9:33:00 PM : 91.77124862670897
2/23/2021 9:34:00 PM : 92.99375228881834
2/23/2021 9:35:00 PM : 94.24249954223632
2/23/2021 9:36:00 PM : 95.34124908447265
2/23/2021 9:37:00 PM : 95.67125091552734
2/23/2021 9:38:00 PM : 96.6949981689453
2/23/2021 9:39:00 PM : 97.19000091552734
2/23/2021 9:40:00 PM : 97.34750366210936
2/23/2021 9:41:00 PM : 97.91750183105468
2/23/2021 9:42:00 PM : 98.397501373291
2/23/2021 9:43:00 PM : 98.58124771118163
2/23/2021 9:44:00 PM : 98.93000183105468
2/23/2021 9:45:00 PM : 99.26000366210937
2/23/2021 9:46:00 PM : 99.22249908447264


In [ ]:
#r "nuget:Microsoft.Azure.CognitiveServices.AnomalyDetector"

Installed package Microsoft.Azure.CognitiveServices.AnomalyDetector version 1.0.0

In [ ]:
using Microsoft.Azure.CognitiveServices.AnomalyDetector;
using Microsoft.Azure.CognitiveServices.AnomalyDetector.Models;
using System.Collections;
using Point = Microsoft.Azure.CognitiveServices.AnomalyDetector.Models.Point;

Request GetSeries(List<(DateTime dt, double val)> list)
{
    var points = list
        // .TakeLast(take)
        .Select(e => new Point(e.dt,e.val)).ToList();
    Console.WriteLine(points);
    return new Request(points, Granularity.Minutely, period: 1); 
}

In [ ]:
async Task<bool> LastDetectSampleAsync(IAnomalyDetectorClient client, Request request)
{

    Console.WriteLine("Detecting the anomaly status of the latest point in the series.");
    LastDetectResponse result = await client.LastDetectAsync(request);
    return result.IsAnomaly;
}

To get Endpoint and Key values, create Anomaly Detector in Azure Portal. https://ms.portal.azure.com/#create/Microsoft.CognitiveServicesAnomalyDetector


To create a cognitive services service and get the key
```

az group create \
    --name cognitive-services-resource-group \
    --location westus2
    
az cognitiveservices account create \
    --name anomaly-detector-resource \
    --resource-group cognitive-services-resource-group \
    --kind AnomalyDetector \
    --sku F0 \
    --location westus2 \
    --yes

az cognitiveservices account keys list \
    --name anomaly-detector-resource \
    --resource-group cognitive-services-resource-group
```

In [ ]:
using System;
using System.IO;
using System.Text;

string path = "config.json";
string readText = File.ReadAllText(path);
var dict = System.Text.Json.JsonSerializer.Deserialize<Dictionary<string, string>>(readText);

string endpoint = dict["endpoint"];
string key = dict["subscriptionkey"];


In [ ]:
IAnomalyDetectorClient client = new AnomalyDetectorClient(new ApiKeyServiceClientCredentials(key))
{
    Endpoint = endpoint
};

In [ ]:
#pragma warning disable CS1702
foreach (var temp in temperatures)
{
    Console.WriteLine(temp.dt + " : " + temp.val);
}
var lastTemp = temperatures.Last();
temperatures.Add((lastTemp.dt.AddMinutes(1) , 200));

var request = GetSeries(temperatures); // The request payload with points from the data file
var isAnomaly = await LastDetectSampleAsync(client, request);
Console.WriteLine($"Anomaly {isAnomaly}");
if (isAnomaly)
{
    using (var ledMatrix = new SenseHatLedMatrixI2c())
    {
            ledMatrix.Fill(Color.Red);
    }
}


2/23/2021 9:32:00 PM : 108.5
2/23/2021 9:33:00 PM : 91.77124862670897
2/23/2021 9:34:00 PM : 92.99375228881834
2/23/2021 9:35:00 PM : 94.24249954223632
2/23/2021 9:36:00 PM : 95.34124908447265
2/23/2021 9:37:00 PM : 95.67125091552734
2/23/2021 9:38:00 PM : 96.6949981689453
2/23/2021 9:39:00 PM : 97.19000091552734
2/23/2021 9:40:00 PM : 97.34750366210936
2/23/2021 9:41:00 PM : 97.91750183105468
2/23/2021 9:42:00 PM : 98.397501373291
2/23/2021 9:43:00 PM : 98.58124771118163
2/23/2021 9:44:00 PM : 98.93000183105468
2/23/2021 9:45:00 PM : 99.26000366210937
2/23/2021 9:46:00 PM : 99.22249908447264
System.Collections.Generic.List`1[Microsoft.Azure.CognitiveServices.AnomalyDetector.Models.Point]
Detecting the anomaly status of the latest point in the series.
Anomaly True


In [ ]:
#pragma warning disable CS1702
using Iot.Device.SenseHat;
using System;
using System.Drawing;
using (var ledMatrix = new SenseHatLedMatrixI2c())
{
        ledMatrix.Fill(Color.Black);
}